# Preparation

In [ ]:
import pandas as pd
import numpy as np
import json
import pickle
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans


data_anime = pd.read_csv('anime_cl.csv')
data_top = json.load(open('anime_top_for_cluster.json'))
clf_model = CatBoostClassifier().load_model('cat_cls.model')
reg_model = CatBoostRegressor().load_model('cat_reg.model')
kmn_model = pickle.load(open('kmeans.model', 'rb'))

# Function for recommendation

In [ ]:
def recomend(usr):
    X = np.array([usr[key] for key in sorted(usr)])
    y = kmn_model.predict([X]).reshape(-1)[0]
    anime_for_usr = np.array(data_top[str(y)])
    anime_f = []
    for anime in anime_for_usr:
        X1 = np.array(data_anime[data_anime['anime_id'] == anime].drop(columns=['anime_id'])).reshape(-1)
        X2 = np.array(pd.DataFrame(usr, index=[0])).reshape(-1)
        X3 = np.array(y).reshape(-1)
        X_f = np.concatenate((X3, X2, X1))
        rate1 = clf_model.predict(X_f).reshape(-1)
        rate2 = reg_model.predict(X_f).reshape(-1)
        if (rate1 + rate2)/2 >= 5:
            anime_f.append(anime)
        if len(anime_f) == 10:
            break
    return anime_f

# Example

In [ ]:
GENRES = data_anime.drop(columns=['anime_id', 'episodes', 'rating', 'members',
                                  'Movie'   , 'Music'   , 'ONA'   , 'OVA', 'TV']).columns.tolist()

In [ ]:
usr = {g: 0 for g in GENRES}

usr['Drama'] = 300
usr['Action'] = 1000
usr['Adventure'] = 1024
usr['Supernatural'] = 512
usr['Comedy'] = 2048

recomendation = recomend(usr)

In [ ]:
anime = pd.read_csv('data/anime.csv')
anime[anime['anime_id'].isin(a)]

,anime_id,name,genre,type,episodes,rating,members
1423,1424,Lupin III: Otakara Henkyaku Daisakusen!!,"Adventure, Comedy, Shounen",Special,1,7.55,2871
1881,2683,Tokyo Majin Gakuen Kenpucho: Tou Dai Ni Maku,"Action, Drama, Fantasy, Horror, Martial Arts, ...",TV,12,7.42,21972
2154,1437,Lupin III: Honoo no Kioku - Tokyo Crisis,"Action, Adventure, Comedy, Mystery, Shounen",Special,1,7.34,3138
2194,1422,Lupin III: $1 Money Wars,"Action, Adventure, Comedy, Shounen",Special,1,7.33,2950
3105,1438,Lupin III: Lupin Ansatsu Shirei,"Action, Adventure, Comedy, Shounen",Special,1,7.08,3006
3256,1431,Lupin III: Ai no Da Capo - Fujiko&#039;s Unluc...,"Action, Adventure, Comedy, Shounen",Special,1,7.04,3279
4063,260,Happy☆Lesson (TV),"Comedy, Drama, Harem, Romance, Slice of Life",TV,13,6.81,17109
5357,1428,Lupin III: Secret File,"Adventure, Comedy, Shounen",OVA,1,6.48,1994
5518,258,Hitsuji no Uta,"Drama, Horror, Vampire",OVA,4,6.43,10574
6005,1427,Lupin III: Ikiteita Majutsushi,"Adventure, Comedy, Shounen",OVA,1,6.28,2697


# User Template

In [ ]:
import json

usr = {g: 0 for g in GENRES}
with open('user_pattern.json', 'w') as f:
    json.dump(usr, f)

In [ ]:
with open('user_pattern.json', 'r') as f:
    usr = json.load(f)